# Obtain ATAC-seq samples from Paola's paper

In [1]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc")

source("script/functions.R"  )


In [2]:
dir.create("pipeline/6.1.footprints"                 , showWarnings = FALSE)
dir.create("pipeline/6.1.footprints/input"           , showWarnings = FALSE)
dir.create("pipeline/6.1.footprints/input/bam"       , showWarnings = FALSE)
dir.create("pipeline/6.1.footprints/input/processing", showWarnings = FALSE)


# Input data
- Samples from Paola's paper (https://www.nature.com/articles/s41588-019-0499-3) and Hi-C paper (https://www.nature.com/articles/s41467-019-08940-5)

In [3]:
uuids_start  = c("5d47b014-54e5-47b3-814c-a1de280537ba", "8068489b-8645-483f-aca6-b87785569490", "429d0e1c-3e06-496a-b819-f31e88a2fed0", "17cfefc0-9748-4f64-ba00-a2e49ec47b65", "917c5313-4a1b-4a27-8e68-df83b7f135f4", "4501297d-1312-42b1-87b7-5f0ead05e635", "d95d608f-a645-4459-bd3b-79999e2d400a", "ddcad17f-2db5-4fbd-bf6a-d3524d00ff13", "f1e0d609-fe23-4a50-9316-e59094738d07", "ffbe88c6-49c0-4fce-8542-56bb2930ee4a", "2ae2913a-a1f0-4844-9a57-79fde6afb7ac", "29f6f3bb-de7e-4204-adf3-19149bf7c143", "bf276c4d-c784-481c-bcc3-a2a1028ad058", "56ffcff4-a3cf-4b70-a19d-8926f2fccb12", "1dc9a2ea-0225-4250-a17e-821dd8eec27a", "5093bad4-50f7-42a2-9d11-62ea6dbef7a9", "5baf0dab-42ba-4419-8d27-bc9406d61d44", "cf5321c1-8ace-4243-a117-d7a3facb213a", "244bb06f-40c8-4f55-a350-39b3b352ca1d", "97ae4780-305e-47dd-ae16-d4da2cc8ce52")
uuid2dir     = data.frame(uuid = uuids_start, infolder = paste("/frazer01/projects/CARDIPS/pipeline/ATAC-Seq/sample", uuids_start, sep = "/"))
uuid2dir     = uuid2dir[ dir.exists(uuid2dir$infolder) == TRUE & uuid2dir$uuid != "5d47b014-54e5-47b3-814c-a1de280537ba", ]
uuid2dir$bam = paste(uuid2dir$infolder, "Aligned.mdup.bam"    , sep = "/")
uuid2dir$bai = paste(uuid2dir$infolder, "Aligned.mdup.bam.bai", sep = "/")


# Merge all BAM files
- link files to here, then merge

In [4]:
invisible(lapply(1:nrow(uuid2dir), function(ii)
{
    uuid = uuid2dir[ii, "uuid"]
    bam  = uuid2dir[ii, "bam" ]
    bai  = uuid2dir[ii, "bai" ]
    
    suppressWarnings(file.symlink(bam, paste("pipeline/6.1.footprints/input/bam", paste(uuid, "bam"    , sep = "."), sep = "/")))
    suppressWarnings(file.symlink(bai, paste("pipeline/6.1.footprints/input/bam", paste(uuid, "bam.bai", sep = "."), sep = "/")))
}))

In [5]:
bamfiles_to_merge = paste("pipeline/6.1.footprints/input/bam", paste(uuid2dir[, "uuid"], "bam", sep = "."), sep = "/")


In [6]:
bam_merged    = paste(getwd(), "pipeline/6.1.footprints/input/processing/merged.bam", sep = "/")
command_merge = paste("samtools", "merge", "--threads", 12, bam_merged, paste(bamfiles_to_merge, collapse = " "))

message(command_merge)

samtools merge --threads 12 /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/merged.bam pipeline/6.1.footprints/input/bam/429d0e1c-3e06-496a-b819-f31e88a2fed0.bam pipeline/6.1.footprints/input/bam/917c5313-4a1b-4a27-8e68-df83b7f135f4.bam pipeline/6.1.footprints/input/bam/d95d608f-a645-4459-bd3b-79999e2d400a.bam pipeline/6.1.footprints/input/bam/ddcad17f-2db5-4fbd-bf6a-d3524d00ff13.bam pipeline/6.1.footprints/input/bam/f1e0d609-fe23-4a50-9316-e59094738d07.bam pipeline/6.1.footprints/input/bam/ffbe88c6-49c0-4fce-8542-56bb2930ee4a.bam pipeline/6.1.footprints/input/bam/29f6f3bb-de7e-4204-adf3-19149bf7c143.bam pipeline/6.1.footprints/input/bam/bf276c4d-c784-481c-bcc3-a2a1028ad058.bam pipeline/6.1.footprints/input/bam/1dc9a2ea-0225-4250-a17e-821dd8eec27a.bam pipeline/6.1.footprints/input/bam/5baf0dab-42ba-4419-8d27-bc9406d61d44.bam pipeline/6.1.footprints/input/bam/cf5321c1-8ace-4243-a117-d7a3facb213a.bam pipeline/6.1.footprints/input/bam/244bb0

In [7]:
bam_sorted    = paste(getwd(), "pipeline/6.1.footprints/input/processing/sorted.bam", sep = "/")
command_sort  = paste("samtools", "sort" , "--threads", 12, "-o", bam_sorted, "-O", "bam", bam_merged)
command_index = paste("samtools", "index", bam_sorted)
message(command_sort )
message(command_index)


samtools sort --threads 12 -o /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/sorted.bam -O bam /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/merged.bam

samtools index /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/sorted.bam



# Run TOBIAS
## Select regions:
- all variants in credible sets (GWAS) +/- 200 bp
- all variants in credible sets (eQTL) +/- 200 bp
- eQTLs: get unique variants upfront:
 - awk '{print $1}' /frazer01/projects/CARDIPS/analysis/cardiac_eqtls/pipeline/3.2.eqtls/eqtls_fine_map/cardiac_eqtls.gene.txt | sort | uniq > /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/credible_sets_eqtls.gene.txt
 
 - awk '{print $1}' /frazer01/projects/CARDIPS/analysis/cardiac_eqtls/pipeline/3.2.eqtls/eqtls_fine_map/cardiac_eqtls.isoform.txt | sort | uniq > /frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/6.1.footprints/input/processing/credible_sets_eqtls.isoform.txt
 

In [8]:
locus2coloc      = fread  ("pipeline/5.4.analyze_coloc_qtl_som_maps//locus2coloc.txt", sep = "\t", header = TRUE, data.table = FALSE)
meta_list        = lapply(locus2coloc$id, function(id){readRDS(paste0("pipeline/5.5.meta_analysis/coloc_data/", id, ".rds"))})
names(meta_list) = locus2coloc$id

In [9]:
cs_gwas = unique(unlist(lapply(meta_list, function(x){x[["finemapped_gwas"]][["credible_set"]]})))

In [10]:
cs_eqtl_gene    = readLines("pipeline/6.1.footprints/input/processing/credible_sets_eqtls.gene.txt"   )
cs_eqtl_isoform = readLines("pipeline/6.1.footprints/input/processing/credible_sets_eqtls.isoform.txt")


In [11]:
cs_eqtl = unique(cs_eqtl_gene, cs_eqtl_isoform)

In [12]:
str(cs_eqtl)
str(cs_gwas)

 chr [1:3915150] "id" "VAR_10_100000625_A_G" "VAR_10_100000645_A_C" ...
 chr [1:185648] "VAR_1_193275821_T_G" "VAR_1_193279932_T_C" ...


In [13]:
get_coord_cs = function(cs, name, distance = 500)
{
    cs  = cs[cs != "null" & cs != "id"]
    out = data.frame(chrom = as.numeric(unlist(lapply(cs, function(x){unlist(strsplit(x, "_"))[[2]]}))),
                     pos   = as.numeric(unlist(lapply(cs, function(x){unlist(strsplit(x, "_"))[[3]]}))),
                     id    = cs
                    )
    out$from  = out$pos - distance
    out$to    = out$pos + distance
    out       = out[order(out$chrom, out$pos),]
    out$chrom = paste0("chr", out$chrom)
    
    bed_unmerged = paste("pipeline/6.1.footprints/input/processing", paste(name, "sorted", "bed", sep = "."), sep = "/")
    bed_out      = paste("pipeline/6.1.footprints/input/processing", paste(name, "merged", "bed", sep = "."), sep = "/")
    command      = paste("bedtools", "merge", "-d", 500, "-i", bed_unmerged, ">", bed_out)
    
    fwrite(out[,c("chrom", "from", "to", "id")], bed_unmerged, sep = "\t", col.names = FALSE, row.names = FALSE)
    
    message(command)
    system (command)
    return(bed_out)
}
bed_gwas = get_coord_cs(cs_gwas, "gwas")
bed_eqtl = get_coord_cs(cs_eqtl, "eqtl")



bedtools merge -d 500 -i pipeline/6.1.footprints/input/processing/gwas.sorted.bed > pipeline/6.1.footprints/input/processing/gwas.merged.bed

bedtools merge -d 500 -i pipeline/6.1.footprints/input/processing/eqtl.sorted.bed > pipeline/6.1.footprints/input/processing/eqtl.merged.bed



In [14]:
dir.create("pipeline/6.1.footprints/tobias"       , showWarnings = FALSE)
dir.create("pipeline/6.1.footprints/tobias/logs"  , showWarnings = FALSE)
dir.create("pipeline/6.1.footprints/tobias/script", showWarnings = FALSE)


In [15]:
run_tobias = function(bed, bam, name, qsub = FALSE)
{
    dir.create(paste("pipeline/6.1.footprints/tobias", name,            sep = "/"), showWarnings = FALSE)
    dir.create(paste("pipeline/6.1.footprints/tobias", name, "correct", sep = "/"), showWarnings = FALSE)
    dir.create(paste("pipeline/6.1.footprints/tobias", name, "score"  , sep = "/"), showWarnings = FALSE)
    dir.create(paste("pipeline/6.1.footprints/tobias", name, "detect" , sep = "/"), showWarnings = FALSE)
    
    folder_correct = paste(getwd(), "pipeline/6.1.footprints/tobias", name, "correct", sep = "/")
    folder_score   = paste(getwd(), "pipeline/6.1.footprints/tobias", name, "score"  , sep = "/")
    folder_detect  = paste(getwd(), "pipeline/6.1.footprints/tobias", name, "detect" , sep = "/")
    
    peak_file      = bed
    bam_file       = bam
    reference_file = "/reference/public/hg19/hg19.fa"
    blacklist_file = "/reference/public/ENCODE/hg19-blacklist.v2.bed"
    motif_file     = "/reference/public/JASPAR/JASPAR2020_CORE_vertebrates_non-redundant_pfms_jaspar.txt"
    
    command_source  = "source activate TOBIAS_ENV"
    command_correct = paste("TOBIAS"     , "ATACorrect",
                            "--bam"      , bam_file,
                            "--genome"   , reference_file,
                            "--peak"     , peak_file,
                            "--blacklist", blacklist_file,
                            "--outdir"   , folder_correct,
                            "--cores"    , 32,
                            "--prefix"   , name)
    
    command_score   = paste("TOBIAS"     , "FootprintScores",
                            "--signal"   , paste0(folder_correct, "/", name, "_corrected.bw"),
                            "--regions"  , peak_file,
                            "--output"   , paste0(folder_score  , "/", name, ".bw"),
                            "--cores"    , 32)
    
    command_detect  = paste("TOBIAS"     , "BINDetect",
                            "--motifs"   , motif_file,
                            "--signals"  , paste0(folder_score  , "/", name, ".bw"),
                            "--genome"   , reference_file,
                            "--peaks"    , peak_file,
                            "--outdir"   , paste0(folder_detect , "/", name),
                            "--cores"    , 32)
    
    commands = c("#$ -pe smp 32", 
                 "#$ -V",
                 "#$ -cwd",
                 paste("#$ -N", name),
                 "#$ -e pipeline/6.1.footprints/tobias/logs",
                 "#$ -o pipeline/6.1.footprints/tobias/logs",
                 "",
                 "source /home/matteo/.bashrc",
                 #command_source,
                 command_correct,
                 command_score,
                 command_detect
                )
    
    sh_file = paste("pipeline/6.1.footprints/tobias/script/run", name, "sh", sep = ".")
    writeLines(commands, con = sh_file, sep = "\n")
    
    if(qsub == TRUE){system(paste("qsub", sh_file))}
}

#run_tobias(bed_gwas, bam_sorted, "gwas", TRUE)
#run_tobias(bed_eqtl, bam_sorted, "eqtl", TRUE)


# After jobs are done, intersect bound/not bound motifs with variants

In [16]:
dir.create("pipeline/6.1.footprints/intersect_motifs", showWarnings = FALSE)


In [17]:
motifs       = fread("/frazer01/projects/CARDIPS/analysis/footprint_eqtl/jaspar/exp_gene.txt", sep = "\t", header = TRUE, data.table = FALSE)
motifs$motif = paste(motifs$base_id, motifs$version, sep = ".")

In [18]:
name        = "gwas"
tfbs_folder = paste("pipeline/6.1.footprints/tobias", name, "detect", name , sep = "/")
tfbs        = data.frame(motif_name = list.dirs(tfbs_folder, recursive = FALSE, full.names = FALSE),
                         motif      = ""
                        )
tfbs$motif  = unlist(lapply(tfbs$motif_name, function(x){unlist(strsplit(x, "_"))[[2]]}))
tfbs        = merge(tfbs, motifs, all = FALSE)

fwrite(tfbs, "pipeline/6.1.footprints/intersect_motifs/tfbs.txt", sep = "\t", col.names = TRUE, row.names = FALSE)


In [19]:
intersect_bed = function(bed_variants, bed_tfbs_list, bed_out)
{
    message(bed_out)
    command = paste("bedtools", "intersect", 
                    "-loj",
                    "-a", bed_variants,
                    paste("-b", bed_tfbs_list, collapse = " "),
                    ">" , bed_out
                   )
    #system (command)
}

intersect_motifs = function(cs, name, tfbs)
{
    message(name)
    tfbs$folder      = paste("pipeline/6.1.footprints/tobias", name, "detect", name, tfbs$motif_name    , sep = "/")
    tfbs$bed_bound   = paste(tfbs$folder, "beds", paste(tfbs$motif_name, name, "bound.bed"  , sep = "_"), sep = "/")
    tfbs$bed_unbound = paste(tfbs$folder, "beds", paste(tfbs$motif_name, name, "unbound.bed", sep = "_"), sep = "/")
    cs               = cs[cs != "null" & cs != "id"]
    out              = data.frame(chrom = as.numeric(unlist(lapply(cs, function(x){unlist(strsplit(x, "_"))[[2]]}))),
                                  pos   = as.numeric(unlist(lapply(cs, function(x){unlist(strsplit(x, "_"))[[3]]}))),
                                  id    = cs
                                 )
    out              = out[order(out$chrom, out$pos),]
    out$chrom        = paste0("chr", out$chrom)
    
    bed_variants = paste("pipeline/6.1.footprints/intersect_motifs", paste(name, "variants", "bed", sep = "."), sep = "/")
    bed_bound    = paste("pipeline/6.1.footprints/intersect_motifs", paste(name, "bound"   , "bed", sep = "."), sep = "/")
    bed_unbound  = paste("pipeline/6.1.footprints/intersect_motifs", paste(name, "unbound" , "bed", sep = "."), sep = "/")
    
    fwrite(out[,c("chrom", "pos", "pos", "id")], bed_variants, sep = "\t")
    
    intersect_bed(bed_variants, tfbs$bed_bound  , bed_bound  )
    intersect_bed(bed_variants, tfbs$bed_unbound, bed_unbound)
}
intersect_motifs(cs_gwas, "gwas", tfbs)
intersect_motifs(cs_eqtl, "eqtl", tfbs)


gwas

pipeline/6.1.footprints/intersect_motifs/gwas.bound.bed

bedtools intersect -loj -a pipeline/6.1.footprints/intersect_motifs/gwas.variants.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/RUNX1_MA0002.2/beds/RUNX1_MA0002.2_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/TFAP2A_MA0003.4/beds/TFAP2A_MA0003.4_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/Ar_MA0007.3/beds/Ar_MA0007.3_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/NR2F1_MA0017.2/beds/NR2F1_MA0017.2_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/CREB1_MA0018.4/beds/CREB1_MA0018.4_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/E2F1_MA0024.3/beds/E2F1_MA0024.3_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/NFIL3_MA0025.2/beds/NFIL3_MA0025.2_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/ELK1_MA0028.2/beds/ELK1_MA0028.2_gwas_bound.bed -b pipeline/6.1.footprints/tobias/gwas/detect/gwas/

# Test associations between fine mapped variants and motifs (GWAS)
## 0. annotate GWAS variants

In [118]:
cs2bin = data.frame(bin  = 1:8, 
                    from = c(0, 1,  5, 10, 20,  50,   100,  10000), 
                    to   = c(1, 5, 10, 20, 50, 100, 10000, 100000))
cs2bin$name = paste(cs2bin$from + 1, cs2bin$to, sep = "-")
cs2bin[1,"name"] = "1"
cs2bin[7,"name"] = "101+"
cs2bin[8,"name"] = "null"

pp2bin = data.frame(bin  = 1:8, 
                    from = c( 99, 95, 90, 80, 50, 10,  1, 0), 
                    to   = c(100, 99, 95, 90, 80, 50, 10, 1))
pp2bin$name = paste(pp2bin$from, pp2bin$to, sep = "-")


In [123]:
annotate_gwas = function(id, meta_list, cs2bin, pp2bin)
{
    #message(id)
    x                  = meta_list[[id]]
    som                = x [["som"            ]]
    traits             = x [["traits_cluster" ]]
    fm                 = x [["finemapped_gwas"]]
    pp                 = fm[["pp"             ]]
    cs                 = fm[["credible_set"   ]]
    som$credible_set   = length(cs)
    bin                = cs2bin[ length(cs) > cs2bin$from & length(cs) <= cs2bin$to,]
    
    if(pp[1, "id"] == "null"){bin = cs2bin[cs2bin$name == "null",]}
    
    colnames(bin)      = paste("credible_set", colnames(bin), sep = ".")
    som                = cbind(som, bin)
    pp                 = pp[ pp$cs == TRUE,]
    pp$bin             = nrow(pp2bin) + 1 - .bincode(pp$pp_snp, breaks = c(sort(pp2bin$from), 100)/100, right = TRUE, include.lowest = TRUE)
    
    return(list(som = som, traits_cluster = traits, pp = pp[,c("id", "pp_snp", "bin")]))
}

#id = "12_113427371_114981406.1"
#annotate_gwas(id, meta_list, cs2bin, pp2bin)

credible_set_list        = lapply(names(meta_list), function(id){annotate_gwas(id, meta_list, cs2bin, pp2bin)})
names(credible_set_list) = names(meta_list)

saveRDS(credible_set_list, "pipeline/6.1.footprints/intersect_motifs/credible_set_list.rds")

In [124]:
credible_sets = as.data.frame(rbindlist(lapply(names(meta_list), function(id)
{
    x           = credible_set_list[[id]][["pp"]]
    x$signal_id = id
    return(x)
})), stringsAsFactors = FALSE)

credible_sets = merge(credible_sets, pp2bin)

## 1. are SNPs with high PP more likely to overlap bound motifs?

In [145]:
name        = "gwas"
bed_bound   = paste("pipeline/6.1.footprints/intersect_motifs", paste(name, "bound"  , "bed", sep = "."), sep = "/")
bed_unbound = paste("pipeline/6.1.footprints/intersect_motifs", paste(name, "unbound", "bed", sep = "."), sep = "/")
bound       = fread(bed_bound  , sep = "\t", header = FALSE, data.table = FALSE)
unbound     = fread(bed_unbound, sep = "\t", header = FALSE, data.table = FALSE)


In [146]:
var2motif = as.data.frame(matrix(0, nrow = length(unique(bound$V4)), ncol = nrow(tfbs), dimnames = list(sort(unique(bound$V4)), tfbs$motif_name)))

for(motif in tfbs$motif_name)
{
    x_bound   = bound  [ bound  $V7 != -1 & bound  $V9 == motif, "V4"]
    x_unbound = unbound[ unbound$V7 != -1 & unbound$V9 == motif, "V4"]
    if(length(x_bound  ) > 0){var2motif[x_bound  , motif] =  1}
    if(length(x_unbound) > 0){var2motif[x_unbound, motif] = -1}
}

cs2motif = merge(credible_sets, var2motif, by.x = "id", by.y = "row.names")

In [151]:
motif2bin = as.data.frame(rbindlist(lapply(tfbs$motif_name, function(motif)
{
    out = as.data.frame(matrix(0, nrow = 1, ncol = nrow(pp2bin), dimnames = list(c(motif), pp2bin$name)))
    for(name in pp2bin$name)
    {
        ids_in  = unique(cs2motif[ cs2motif$name == name & cs2motif[,motif] == 1, "id"])
        ids_all = unique(cs2motif[ cs2motif$name == name & cs2motif[,motif] != 0, "id"])
        out[motif, name] = length(ids_in) / length(ids_all)
    }
    return(out)
})), stringsAsFactors = FALSE)

rownames(motif2bin) = tfbs$motif_name


In [158]:
var2bound = data.frame(id      = rownames(var2motif), 
                       bound   = unlist(lapply(1:nrow(var2motif), function(ii)
                       {
                           x   = as.numeric(var2motif[ii,])
                           out = ifelse(test = length(x[x == 1]) > 0, yes = 1, no = 0)
                           return(out)
                       })),
                       unbound = unlist(lapply(1:nrow(var2motif), function(ii)
                       {
                           x   = as.numeric(var2motif[ii,])
                           out = ifelse(test = (length(x[x == 1]) == 0)&(length(x[x == -1]) > 0), yes = 1, no = 0)
                           return(out)
                       }))
                      )

var2bound = merge(credible_sets, var2bound)

In [169]:
bound2bin         = do.call(data.frame, aggregate(bound ~ bin + from + to + name, data = var2bound, FUN = function(x) {c(n_bound = sum(x), n = length(x))}))
bound2bin$bound_x =  bound2bin$bound.n_bound / bound2bin$bound.n * 100

In [170]:
bound2bin

bin,from,to,name,bound.n_bound,bound.n,bound_x
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
8,0,1,0-1,12740,150350,8.473562
7,1,10,1-10,3072,33836,9.079087
6,10,50,10-50,673,5164,13.032533
5,50,80,50-80,85,537,15.828678
4,80,90,80-90,36,169,21.301775
3,90,95,90-95,14,101,13.861386
2,95,99,95-99,21,115,18.260870
1,99,100,99-100,53,267,19.850187


In [174]:
motif99       = tfbs
motif99$bound = colSums(cs2motif[ cs2motif$id %in% var2bound[ var2bound$bin == 1 & var2bound$bound == 1, "id"], tfbs$motif_name])


In [176]:
motif99[order(motif99$bound, decreasing = TRUE),]

,motif,motif_name,base_id,version,id,name,prot_acc,gene_id,gene_name,bound
,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
75,MA0162.4,EGR1_MA0162.4,MA0162,4,10653,EGR1,P18146,ENSG00000120738.8_4,EGR1,8
127,MA0599.1,KLF5_MA0599.1,MA0599,1,10761,KLF5,Q13887,ENSG00000102554.14_5,KLF5,6
194,MA0753.2,ZNF740_MA0753.2,MA0753,2,10951,ZNF740,Q8NDX6,ENSG00000139651.11_4,ZNF740,6
278,MA1107.2,KLF9_MA1107.2,MA1107,2,11334,KLF9,Q13886,ENSG00000119138.4_4,KLF9,6
348,MA1513.1,KLF15_MA1513.1,MA1513,1,11768,KLF15,Q9UIH9,ENSG00000163884.4_5,KLF15,6
402,MA1596.1,ZNF460_MA1596.1,MA1596,1,11851,ZNF460,Q14592,ENSG00000197714.9_4,ZNF460,6
14,MA0039.4,KLF4_MA0039.4,MA0039,4,11333,KLF4,O43474,ENSG00000136826.15_7,KLF4,5
31,MA0073.1,RREB1_MA0073.1,MA0073,1,9301,RREB1,Q92766,ENSG00000124782.20_9,RREB1,5
189,MA0746.2,SP3_MA0746.2,MA0746,2,10943,SP3,Q02447,ENSG00000172845.16_6,SP3,5


In [110]:
credible_sets[ credible_sets$id == "null",]

,bin,id,pp_snp,signal_id,from,to,name
,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
19,1,null,0.9958952,4_91137236_91403287.1,99,100,99-100
26,1,null,0.9963544,5_82658946_82957040.1,99,100,99-100
32,1,null,0.9920117,5_118432034_118961227.1,99,100,99-100
42,1,null,0.9925802,18_0_417376.1,99,100,99-100
44,1,null,0.9940421,12_16325160_16593904.1,99,100,99-100
61,1,null,0.9902784,16_62671591_62992431.1,99,100,99-100
70,1,null,0.9912676,8_73326274_73584874.1,99,100,99-100
77,1,null,0.9936481,17_68319330_68611145.1,99,100,99-100
102,1,null,0.9910859,1_10970919_11460957.1,99,100,99-100
